## <center> Tubular and ribbon surface defined by Frenet-Serre equations of their spine curve</center>

**1. Tubular surface**

A tubular surface  is generated by a 3D curve, called spine, and a moving circle of radius r, with center on the spine and  included in planes orthogonal to curve.

Tubular surfaces are associated to spines that are biregular, that is, they have a $C^2$ parameterization, $c:[a,b]\to \mathbb{R}^3$, with non-null and non-collinear velocity, and  acceleration, i.e.
$\dot{c}(t)\times \ddot{c}(t)\neq 0$.
A tube of prescribed [curvature](https://en.wikipedia.org/wiki/Curvature#Curvature_of_space_curves) and [torsion](https://en.wikipedia.org/wiki/Torsion_of_a_curve) is defined by a spine parameterized by the arc length, i.e. by
a parameterization, $c(s)$, with constant speed, $||\dot{c}(s)||=1$, and non-null acceleration, $\ddot{c}(s)\neq 0$, for all $s$.

The given curvature and torsion, $\kappa(s)$,  $\tau(s)$, define the Frenet-Serre equations:
$$\begin{array}{lll}
    \dot{e}_1(s)&=&\kappa(t)e_2(s)\\
    \dot{e}_2(s)&=&-\kappa(s)e_1(s)+\tau(s)e_3(s)\\
    \dot{e}_3(s)&=&-\tau(s)e_2(s),\\
    \end{array} $$
    
where $e_1(s), e_2(s), e_3(s)$ are  respectively the unit vectors of tangent, principal normal and binormal along the curve.

Frenet-Serre equations completed with the equation $ \dot{c}(s)=e_1(s)$ define a system of ordinary differential equations, with  12 equations and 12 unknown functions. The last three
coordinates of a solution represent the  discretized curve,  $c(s)$, starting from an initial point, with a prescribed Frenet frame at that point.

We define below a tubular surface with highly oscillating curvature and constant torsion of the spine.

A  tubular surface having as spine  a curve, $c(s)$, parameterized by the arclength, is defined as follows:
$r(s,u)=c(s)+\varepsilon(e_2(s)cos(u)+e_3(s)sin(u))$, $0<\varepsilon <<1$, $u\in[0, 2\pi]$.
$\varepsilon$ is the radius of circles orthogonal to the spine.

To each vectorial equation of the above system one associates three equations for the coordinates of the corresponding vectors:
$$\begin{array}{lll}
e_1&=&(u[1], u[2], u[3])\\
e_2&=&(u[4], u[5], u[6])\\
e_3&=&(u[7], u[8], u[9])\end{array}$$


$(u[10], u[11], u[12])$,  are the coordinates of points  on the curve.

Calculate the spine curve of given curvature and torsion function, as well as the vectors of the Frenet frame along this curve, by integrating a system of 12 ODEs:

In [ ]:
using DifferentialEquations
using StaticArrays
using PlotlyJS
include("src/plotlyju.jl");
include("src/PlotlyColorSchemes.jl");

In [ ]:
function Frenet_eq!(du, u, p, s)  #system of ODEs 
    du[1] = κ(s)*u[4]
    du[2] = κ(s)*u[5]
    du[3] = κ(s)*u[6]
    du[4] = -κ(s)*u[1]+τ(s)*u[7]
    du[5] = -κ(s)*u[2]+τ(s)*u[8]
    du[6] = -κ(s)*u[3]+τ(s)*u[9]
    du[7] = -τ(s)*u[4]
    du[8] = -τ(s)*u[5]
    du[9] = -τ(s)*u[6]
    du[10] = u[1]
    du[11] = u[2]
    du[12] = u[3]
end

In [ ]:
κ(s) = 3*sin(s/10)*sin(s/10) #curvature function
τ(s) = 0.35 #constant torsion

u0 = [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0] #initial condition for the system of ODE
tspan = (0.0,  150) #time interval of integration
prob = ODEProblem(Frenet_eq!, u0, tspan)
sol = solve(prob);
s = 0:0.15:150; #discrete arclength to interpolate the solution 
vect_sol  = sol.(s);

# convert from vector of vectors to vector of static 12-vectors
vect_sol = Vector{SVector{12, Float64}}(vect_sol)
#convert and reshape the solution to get a matrix of size (length(s), 12)
matr_sol = reshape(reinterpret(Float64, vect_sol), (12, length(vect_sol)))';  

e₂ =  matr_sol[:, 4:6]
e₃ = matr_sol[:, 7:9]
spine = matr_sol[:, 10:12]
#evaluate points x, y, z on the tubular surface 
lu = 100
u = collect(range(0, stop= 2π, length = lu))
epsilon = 0.2
# find the coordinates of the surface parameterization
#r(s,u)=c(s)+epsilon(e_2(s)cos(u)+e_3(s)sin(u))
x, y, z = [spine[:, k] .* ones(lu)' + 
           epsilon*(e₂[:, k] .* cos.(u)' + 
           e₃[:, k] .* sin.(u)') for k in 1:3];

Plot the tubular surface colored according to the curvature of the spine, i.e. the color of curvature of the center of each circle perpendicular to the spine
is assigned to the points of that circle, as well:

In [ ]:
tube = surface(x=x, y=y, z=z, 
               surfacecolor = κ.(s) .* ones(lu)', 
               colorbar_len=0.5)

pl = plot(tube, Layout(width=800, height= 800,
                       scene_camera_eye=attr(x=2, y=2, z=0.6)),
                       style=plotlyju)

![tubular](images/tubular_surf.png)

The same tube with the height (z-values) colormapped to the default colorscale (Plasma):

![tube-z-color](images/tube-z-color.png)

**2. Ribbon surface**

Ribbon surfaces can be used for visualizing  protein structure. From the above approach we can derive such a surface.
Namely, once the spine curve is defined by Frenet-Serre equations, a ribbon surface having this spine,  is parameterized by:
$$r(s,t)= c(s)+te_3(s), \quad t\in[-a,a], a>0$$


Our discrete ribbon is parameterized by (xr, yr, zr), where:

In [ ]:
a= 0.15
t = LinRange(-a, a, 10)
xr = spine[:, 1] * ones(size(t))'+ e₃[:,1]* t'
yr = spine[:, 2] * ones(size(t))'+ e₃[:,2]* t'
zr = spine[:, 3] * ones(size(t))'+ e₃[:,3]* t';

In [ ]:
ribbon = surface(x=xr, y=yr, z=zr,
                 colorscale=pl_viridis,
                 colorbar_len=0.5)
fig = Plot(ribbon, Layout(width=800, height= 800,
                       scene_camera_eye=attr(x=2, y=2, z=0.6)),
                       style=plotlyju)


![ribbon](images/ribbon-surf.png)